In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

import os
import json
import collections
import math

In [3]:
general_classes = ['airplane', 'antelope', 'bear', 'bicycle', 'bird', 'bus', 'car',
                  'cattle', 'dog', 'domestic_cat', 'elephant', 'entity', 'fox', 'giant_panda',
                  'hamster', 'horse', 'lion', 'lizard', 'monkey', 'motorcycle', 'rabbit',
                  'red_panda', 'sheep', 'snake', 'squirrel', 'tiger', 'train', 'turtle',
                  'watercraft', 'whale', 'zebra']

In [4]:
actual_class = pd.read_json('file://localhost/Users/sam/imagenet-vid-robust-example-master/imagenet-vid-robust/metadata/big_labels.json',typ='series')

In [5]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [6]:
def get_actual_class(file,actual):
    if '\\' in file:
        file = file.replace('\\','/')
        
    key = os.path.basename(os.path.dirname(os.path.dirname(file))) +'/' \
    + os.path.basename(os.path.dirname(file)) +'/'+ os.path.basename(file)
    return actual[key]

In [7]:
def get_choice(data_frame,both=False):
    confidence = 0
    choice = ''
    for key, val in data_frame.items():
        if val > confidence:
            choice = key
            confidence = val
            
    if both:
        return (choice, confidence)
    else:
        return choice

In [8]:
def get_accuracy(data_frame):
    right = 0
    wrong = 0
    
    for key in data_frame:
        if get_choice(data_frame[key]) == get_actual_class(key,actual_class):
            right += 1
        else:
            wrong += 1
    
    accuracy = right/len(data_frame.keys())*100
    print(f'Regular accuracy - right: {right}, Wrong: {wrong}, Percent Correct: {accuracy}')
    
    return accuracy

In [9]:
def new_data_frame(data_frame,comparison):
    new_df = {}
    
    for image in data_frame:
        choice, num = get_choice(data_frame[image], both=True)
        
        new_df[image] = {}
        new_df[image]['actual_class'] = get_actual_class(image,comparison)
        new_df[image]['class'] = choice
        new_df[image]['confidence'] = num
                
    return new_df

In [10]:
def get_anchor_frames(df):
    folder_set = [os.path.dirname(os.path.dirname(os.path.dirname(list(df.keys())[0]))),[],[]]
    index = 0
    
    def full_name():
        path = ''
        for part in folder_set:
            path += part[math.floor(len(part)/2)] if isinstance(part,list) else part
        return path 
    
    for image in df:
        index += 1
        
        dir_name = '/' + os.path.basename(os.path.dirname(os.path.dirname(image))) + '/' \
        + os.path.basename(os.path.dirname(image)) + '/'
        
        basename = os.path.basename(image)
        
        if (dir_name not in folder_set[1] and len(folder_set[1]) != 0) or index == len(df.keys()):
            if index == len(df.keys()):
                folder_set[1].append(dir_name)
                folder_set[2].append(basename)
            
            for file in folder_set[2]:
                key = folder_set[0]+folder_set[1][0]+file
                if key == full_name():
                    df[image]['anchor'] = True
                else:
                    df[image]['anchor'] = False
            
            folder_set[1] = []
            folder_set[2] = []

        folder_set[1].append(dir_name)
        folder_set[2].append(basename)
        
    return df

In [11]:
def create_anchor_frame_df(data_frame):
    anchor_df = {}
    for image in data_frame:
        if data_frame[image]['anchor']:
            anchor_df[image] = data_frame[image]['class']
            
    return anchor_df

In [12]:
def get_anchor_accuracy(data_frame):
    right = 0
    wrong = 0
    
    for key in data_frame:
        if data_frame[key] == get_actual_class(key,actual_class):
            right += 1
        else:
            wrong += 1
            
    accuracy = right/len(data_frame.keys())*100
    print(f'Anchor accuracy - right: {right}, Wrong: {wrong}, Percent Correct: {accuracy}')
    
    return accuracy

In [14]:
models =['VGG19','SqueezeNet1.0','AlexNet']

for model in models:
    print(model)
    
    df = pd.read_json(f'file://localhost/Users/sam/save-results/{model}_full.json')
    
    reduced_df = new_data_frame(df[:],actual_class)
    reduced_df = pd.DataFrame(data=get_anchor_frames(reduced_df))
    anchor_df = create_anchor_frame_df(reduced_df)
    
    print(math.floor(get_anchor_accuracy(anchor_df)-get_accuracy(df)*1000)/1000)
    print('\n')

VGG19
Anchor accuracy - right: 8506, Wrong: 12566, Percent Correct: 40.366362946089595
Regular accuracy - right: 8949, Wrong: 13230, Percent Correct: 40.348978763695385
-40.309


SqueezeNet1.0


URLError: <urlopen error [Errno 2] No such file or directory: '/Users/sam/save-results/SqueezeNet1.0_full.json'>